<a href="https://colab.research.google.com/github/Viaxiz/Machine-Learning-Study/blob/main/%E5%8A%A8%E6%89%8B%E5%AD%A6%E6%B7%B1%E5%BA%A6%E5%AD%A6%E4%B9%A001.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch

In [6]:
 X = torch.arange(12,dtype = torch.float32).reshape((3,4))
 Y = torch.tensor([[0,1,2,3],[0,1,2,3],[-1,-2,-3,-4]])


In [9]:
X,Y

(tensor([[ 0.,  1.,  2.,  3.],
         [ 4.,  5.,  6.,  7.],
         [ 8.,  9., 10., 11.]]), tensor([[ 0,  1,  2,  3],
         [ 0,  1,  2,  3],
         [-1, -2, -3, -4]]))

张量运算符之逻辑运算

In [7]:
X == Y

tensor([[ True,  True,  True,  True],
        [False, False, False, False],
        [False, False, False, False]])

In [8]:
X < Y

tensor([[False, False, False, False],
        [False, False, False, False],
        [False, False, False, False]])

In [10]:
X > Y

tensor([[False, False, False, False],
        [ True,  True,  True,  True],
        [ True,  True,  True,  True]])

广播机制

In [16]:
a = torch.arange(3).reshape(3,1)
b = torch.arange(2).reshape(1,2)
a,b

(tensor([[0],
         [1],
         [2]]), tensor([[0, 1]]))

In [17]:
a + b 

tensor([[0, 1],
        [1, 2],
        [2, 3]])

数据预处理

In [18]:
import os

In [20]:
os.makedirs(os.path.join('..', 'data'), exist_ok=True)
data_file = os.path.join('..', 'data', 'house_tiny.csv')
with open(data_file, 'w') as f:
    f.write('NumRooms,Alley,Price\n')  # 列名
    f.write('NA,Pave,127500\n')  # 每行表示一个数据样本
    f.write('2,NA,106000\n')
    f.write('4,NA,178100\n')
    f.write('NA,NA,140000\n')

In [21]:
import pandas as pd

In [22]:
data = pd.read_csv(data_file)
print(data)

   NumRooms Alley   Price
0       NaN  Pave  127500
1       2.0   NaN  106000
2       4.0   NaN  178100
3       NaN   NaN  140000


In [26]:
inputs,outputs = data.iloc[:,0:2],data.iloc[:,2]
inputs = inputs.fillna(inputs.mean())
print(inputs)

   NumRooms Alley
0       3.0  Pave
1       2.0   NaN
2       4.0   NaN
3       3.0   NaN


<ipython-input-26-ae78360876f7>:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  inputs = inputs.fillna(inputs.mean())


In [27]:
outputs

0    127500
1    106000
2    178100
3    140000
Name: Price, dtype: int64

对于inputs中的类别值或离散值，我们将“NaN”视为一个类别。 由于“巷子类型”（“Alley”）列只接受两种类型的类别值“Pave”和“NaN”， pandas可以自动将此列转换为两列“Alley_Pave”和“Alley_nan”。 巷子类型为“Pave”的行会将“Alley_Pave”的值设置为1，“Alley_nan”的值设置为0。 缺少巷子类型的行会将“Alley_Pave”和“Alley_nan”分别设置为0和1。

In [29]:
inputs = pd.get_dummies(inputs, dummy_na = True)
print(inputs)

   NumRooms  Alley_Pave  Alley_nan
0       3.0           1          0
1       2.0           0          1
2       4.0           0          1
3       3.0           0          1


转换成张量格式

In [31]:
type(inputs),type(outputs)

(pandas.core.frame.DataFrame, pandas.core.series.Series)

In [33]:
X,y = torch.tensor(inputs.values),torch.tensor(outputs.values)
X,y

(tensor([[3., 1., 0.],
         [2., 0., 1.],
         [4., 0., 1.],
         [3., 0., 1.]], dtype=torch.float64),
 tensor([127500, 106000, 178100, 140000]))

拼接起来

In [36]:
torch.cat((X,y.reshape(4,1)),dim = 1)

tensor([[3.0000e+00, 1.0000e+00, 0.0000e+00, 1.2750e+05],
        [2.0000e+00, 0.0000e+00, 1.0000e+00, 1.0600e+05],
        [4.0000e+00, 0.0000e+00, 1.0000e+00, 1.7810e+05],
        [3.0000e+00, 0.0000e+00, 1.0000e+00, 1.4000e+05]], dtype=torch.float64)

In [60]:
os.makedirs(os.path.join('..','data'),exist_ok=True)
data_made_own = os.path.join('..','data','animal.csv')
with open(data_made_own,'w',encoding='utf8') as f:
    f.write('Name,Type,Number,Weight\n')  #列名
    f.write('Tiger,Cat,2,1200\n')
    f.write('Lion,Cat,10,9000\n')
    f.write('Pig,NA,12,30000\n')
    f.write('Cat,Cat,100\n')
    f.write('Dog,Dog,5,280\n')
    f.write('Dragon,NA,1,NA\n')
    f.write('Duck,Bird,20,300\n')
    f.write('Chicken,Bird,15,NA\n')

data_own = pd.read_csv(data_made_own)
# print(data_own)
data_own

,Name,Type,Number,Weight
0,Tiger,Cat,2,1200.0
1,Lion,Cat,10,9000.0
2,Pig,NaN,12,30000.0
3,Cat,Cat,100,NaN
4,Dog,Dog,5,280.0
5,Dragon,NaN,1,NaN
6,Duck,Bird,20,300.0
7,Chicken,Bird,15,NaN


In [61]:
print(len(data_own))

8


In [62]:
# 删除缺省值最多的列
def drop_col():
  n = len(data_own)
  temp = 0 
  for col in('Name','Type','Number','Weight'):
    cnt = data_own[col].count()
    # print("col=", col, ":", cnt,"\n")
    if(n - cnt) > temp:
      temp = n - cnt
      drop_name = col
  data_own.drop(drop_name,axis=1,inplace=True)

drop_col()
data_own #html格式展示

col= Name : 8 

col= Type : 6 

col= Number : 8 

col= Weight : 5 



,Name,Type,Number
0,Tiger,Cat,2
1,Lion,Cat,10
2,Pig,NaN,12
3,Cat,Cat,100
4,Dog,Dog,5
5,Dragon,NaN,1
6,Duck,Bird,20
7,Chicken,Bird,15
